# YAML Tag Extensions

In [1]:
from pathlib import Path
import sys
sys.path.insert(0, str(Path("..").resolve()))
import yamx

## Tag: !env

Load environmental variables using this tag. The variables are automatically cast to `int`, `float`, or `str`. Use quotes (`'` or `"`) to prevent this behavior.

In [3]:
%env VAR="1.23e02"
print(yamx.loads("var: !env:VAR"))
%env VAR=1.23e02
print(yamx.loads("var: !env:VAR"))

env: VAR="1.23e02"
int failed
float failed
{'var': '1.23e02'}
env: VAR=1.23e02
int failed
{'var': 123.0}


## Tag: !from

Parts or whole other config files can be inserting using the `!from:` tag.

The string that follows the tag is referred to as "navigation string".
A navigation string must contain the absolute or relative path to a configuration file. An absolute path is headed by `/`, e.g.:
- absolute: `/this/path/is/absolute.yaml`
- relative: `this/path/is/relative.yaml`

Optionally, a navigation path can include a "tree" navigation that points to a value inside a config file. Let `/path/to/dir/filename.yaml` be the absolute path to a configurations file with the content:

```yaml
navigate:
  to: 
    value: 0
```

The `0` can be retrieved by appending `navigate/to/value` to the file path: `path/to/dir/filename.yaml/navigate/to/value`.

YAML extensions can be omitted inside the navigation string, so that the following examples are equal:
- `path/to/dir/filename.yaml/navigate/to/value`
- `path/to/dir/filename/navigate/to/value`

The following example provides a summary of the steps that are performed to parse the tag: `!from:path/to/dir/filename/navigate/to/value`.

1. get navigation string: `path/to/dir/filename/navigate/to/value`
2. find information:
    - directory: `path/to/dir`
    - filename: `filename.yaml` (the `.yaml` extension is found automatically)
    - tree: `["navigate", "to", "value"]`
3. load file `path/to/dir/filename.yaml`
4. use tree values to navigate to value

### Example 1: Automatically find extensions

In [ ]:
%%writefile constants.yaml
value: 123
list:
  - 0
  - 
    - 1
    - 2

In [ ]:
file_text = """\
- !from:constants.yaml
- !from:constants
"""

yamx.loads(file_text)

### Example 2: Load value from child config file into the parent config

In [ ]:
file_text = """\
map: 
  !from:constants/value
"""

yamx.loads(file_text)

### Example 3: Navigating child config file tree with sequence elements

In [ ]:
file_text = """\
!from:constants/list       # insert complete list: [0, [1, 2]]
- !from:constants/value    # insert a value: 123
- !from:constants/list/1/0 # access list element: 1
"""

yamx.loads(file_text)

## Tag: !import

Define a demo class in `module.py` so that it can be imported.

In [ ]:
%%writefile example_module.py
class ExampleClass:
    text = "I am not initialized."
    def __init__(self, *args, value="default", **kwargs):
        print("init ExampleClass with", args, kwargs)
        self.value = value
        self.text = "I am initialized."

    def set_value(self, value=None):
        if value is not None:
            self.value = value
        print("call self.set_value with", value)
        return self

    @classmethod
    def from_example(cls, value):
        print("call DemoClass.from_example with", value)
        return cls

In [ ]:
file_text = """\
class: !import:example_module.ExampleClass
"""

my_class = yamx.loads(file_text)["class"]
my_class.text

## Tag: !init

In [ ]:
file_text = """\
class: !init:example_module.ExampleClass
"""

my_class = yamx.loads(file_text)["class"]
my_class.text

In [ ]:
file_text = """\
class: !init:example_module.ExampleClass
- arg0
- arg1
- kwarg0: 0
- kwarg1: 1
"""

my_class = yamx.loads(file_text)["class"]
my_class.text

## Tag: !chain with !call

In [ ]:
file_text = """\
value:
  !chain
  - !init:example_module.ExampleClass
  - !call:set_value
    - 123
  - !call:value
"""

yamx.loads(file_text)

In [ ]:
file_text = """\
class:
  !chain
  - !import:example_module.ExampleClass
  - !call:from_example
    - 123
"""

my_class = yamx.loads(file_text)["class"]
my_class.text